In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
import pandas as pd
import math
load_dotenv()
base_path = Path.home() / Path(os.environ.get("DATA_PATH"))
print(base_path)

/home/aayush/rsys_data


In [2]:
df = pd.read_csv(
   base_path/ Path("behaviors.tsv"), 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset originally consist of {len(df)} number of interactions.")


df.head()

The dataset originally consist of 2232748 number of interactions.


,impressionId,userId,timestamp,click_history,impressions
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...


In [3]:
news = pd.read_csv(
    base_path/ Path("news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
news

The article data consist in total of 101527 number of articles.


,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


In [4]:
categories = news['category'].unique()
news["num_category"] = news["category"].factorize()[0]
article_category = news.set_index("itemId")["num_category"].to_dict()

Split the impressions in two columns clicked items and presented slate

In [5]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings]
    return click,non_click


df['click'], df['presented_slate'] = zip(*df['impressions'].map(process_impression))
df

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1,[N47925],"[N103852, N53474, N127836, N47925]"
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...,[N114935],"[N38902, N76434, N71593, N100073, N108736, N30..."
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...,[N86258],"[N76209, N48841, N67937, N62235, N6307, N34254..."
...,...,...,...,...,...,...,...
2232743,2232744,U316192,11/13/2019 6:50:02 PM,N122359 N37069 N95876 N28787 N73408 N11266 N61321,N113723-0 N123683-1 N5287-0 N76677-0 N53474-0,[N123683],"[N113723, N123683, N5287, N76677, N53474]"
2232744,2232745,U451238,11/12/2019 8:54:06 AM,N12575 N93816 N71643 N87236 N87236,N18861-0 N20990-0 N43085-0 N7937-1,[N7937],"[N18861, N20990, N43085, N7937]"
2232745,2232746,U151246,11/13/2019 12:42:51 PM,N27587 N49668,N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,"[N39887, N110709, N24001, N127572, N10285]","[N39887, N22811, N110709, N1923, N24001, N7667..."
2232746,2232747,U330725,11/12/2019 1:22:57 PM,N121944 N91510 N42280 N60061 N63032 N125223 N4...,N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...,[N88808],"[N18947, N88808, N10012, N38902, N33078, N1003..."


In [6]:
valid_article_ids = set(news['itemId'])
print(len(news['itemId']))
def filter_click_list(click_list):
    return [item for item in click_list if item in valid_article_ids]
def filter_click_history_list(click_list):
    if pd.notna(click_list['click_history']):
     return [item for item in click_list['click_history'].split() if item in valid_article_ids]
    else:
     return []
behaviors_df=df

behaviors_df['click'] = behaviors_df['click'].apply(filter_click_list)
behaviors_df['presented_slate'] = behaviors_df['presented_slate'].apply(filter_click_list)
behaviors_df['click_history'] = behaviors_df.apply(filter_click_history_list,axis=1)
# behaviors_df[behaviors_df['click'].apply(lambda x: any(item in valid_article_ids for item in x))]
# filtered_behaviors_df = filtered_behaviors_df[filtered_behaviors_df['presented_slate'].apply(lambda x: any(item in valid_article_ids for item in x))]
filtered_behaviors_df = behaviors_df[behaviors_df['click'].apply(lambda x: len(x) > 0)]
filtered_behaviors_df

101527


,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,3,U532401,11/13/2019 11:23:03 AM,"[N128643, N87446, N122948, N9375, N82348, N129...",N103852-0 N53474-0 N127836-0 N47925-1,[N47925],"[N103852, N53474, N127836, N47925]"
3,4,U593596,11/12/2019 12:24:09 PM,"[N31043, N39592, N4104, N8223, N114581, N92747...",N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...,[N114935],"[N38902, N76434, N71593, N100073, N108736, N30..."
4,5,U239687,11/14/2019 8:03:01 PM,"[N65250, N122359, N71723, N53796, N41663, N414...",N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...,[N86258],"[N76209, N48841, N67937, N62235, N6307, N34254..."
...,...,...,...,...,...,...,...
2232743,2232744,U316192,11/13/2019 6:50:02 PM,"[N122359, N37069, N95876, N28787, N73408, N112...",N113723-0 N123683-1 N5287-0 N76677-0 N53474-0,[N123683],"[N113723, N123683, N5287, N76677, N53474]"
2232744,2232745,U451238,11/12/2019 8:54:06 AM,"[N12575, N93816, N71643, N87236, N87236]",N18861-0 N20990-0 N43085-0 N7937-1,[N7937],"[N18861, N20990, N43085, N7937]"
2232745,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,"[N39887, N110709, N24001, N127572, N10285]","[N39887, N22811, N110709, N1923, N24001, N7667..."
2232746,2232747,U330725,11/12/2019 1:22:57 PM,"[N121944, N91510, N42280, N60061, N63032, N125...",N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...,[N88808],"[N18947, N88808, N10012, N38902, N33078, N1003..."


Single click data

In [7]:
filtered_df_choice_data = filtered_behaviors_df[
    (df['click'].apply(lambda x: len(x) == 1 or (len(x) == 0 and isinstance(x, list))))
].reset_index()

More thna single click

In [10]:
filtered_df_rl = filtered_behaviors_df[
    (df['click'].apply(lambda x: len(x) != 1 or (len(x) == 0 and isinstance(x, list))))
].reset_index()

In [11]:
filtered_df_rl=filtered_df_rl.explode('click', ignore_index=True)
filtered_df_rl=filtered_df_rl.drop(columns=['index'])

In [12]:
filtered_df_rl

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1769833,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769834,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769835,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769836,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [14]:
base_path = Path("/home/aayush/rsys_data")
save_dir = base_path / "rsys_2025"
save_dir.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist

feather_path = save_dir / "interactions.feather"
filtered_df_rl.to_feather(feather_path)